# Data Analysis of the Health Care Provider Dataset

In this notebook, we analyse the data and network structure of the Health Care Provider Dataset from kaggle. This mainly concerns the possibilities to split the data for training and testing. 

## Loading and inspecting the data

The data provided on kaggle consists of four different datasets.

In [ ]:
import pandas as pd

In [ ]:
labels = pd.read_csv('../data/HCP/Train.csv')
beneficiary = pd.read_csv('../data/HCP/Train_Beneficiarydata.csv')
inpatient = pd.read_csv('../data/HCP/Train_Inpatientdata.csv')
outpatient = pd.read_csv('../data/HCP/Train_Outpatientdata.csv')

In [ ]:
labels['Label'] = [0 if x=='No' else 1 for x in labels['PotentialFraud']]
labels.set_index('Provider', inplace=True)
labels = labels[['Label']]
labels.head()

In [ ]:
beneficiary.head()

In [ ]:
inpatient.head()

In [ ]:
outpatient.head()

## Feature engineering

The labels are available for providers. These nodes do not have any features, so we will calculate the following features:
* number of claims;
* avg reembursed;
* std reembursed;
* number of claims per beneficiary.

These summarise some key characteristics of the providers, that might help the algorithm uncover fraudulent behaviour. 

In [ ]:
columns_to_select = ['Provider', 'BeneID', 'InscClaimAmtReimbursed']
edges = pd.concat([inpatient[columns_to_select], outpatient[columns_to_select]])
edges.head()

In [ ]:
number_of_claims = edges[['Provider', 'InscClaimAmtReimbursed']].groupby(['Provider']).count()
number_of_claims.columns = ['number_of_claims']

average_claims = edges[['Provider', 'InscClaimAmtReimbursed']].groupby(['Provider']).mean()
average_claims.columns = ['average_claims']

std_claims = edges[['Provider', 'InscClaimAmtReimbursed']].groupby(['Provider']).std()
std_claims.columns = ['std_claims']

number_of_beneficiaries = edges[['Provider', 'BeneID']].drop_duplicates().groupby(['Provider']).count()
number_of_beneficiaries.columns = ['number_of_beneficiaries']

provider_features = pd.concat([number_of_claims, average_claims, std_claims, number_of_beneficiaries], axis=1)
provider_features.index.rename('Target', inplace=True)
provider_features.head()


In [ ]:
provider_features.isnull().sum()

In [ ]:
provider_features.fillna(0, inplace=True)

In [ ]:
beneficiary['Deceased'] = 1
beneficiary.loc[beneficiary['DOD'].isna(), 'Deceased'] = 0
beneficiary_features = beneficiary.drop(columns=["DOD"])

beneficiary_features['RenalDiseaseIndicator'] = [1 if x == 'Y' else 0 for x in beneficiary['RenalDiseaseIndicator']]

beneficiary_features['DOB'] = beneficiary_features['DOB'].str.split('-').str[0].astype(int)
beneficiary_features.set_index('BeneID', inplace=True)
beneficiary_features.head()

beneficiary_features.isnull().sum()

## Network construction

For the network, we work exclusively with providers and beneficiaries. We will look at the connected components in an attempt to find a meaningful train-test split. 

In [ ]:
edges

In [ ]:
import networkx as nx
G = nx.Graph()

for node in provider_features.index:
    G.add_nodes_from([(node, {'label': 'provider'})])
for node in beneficiary_features.index:
    G.add_nodes_from([(node, {'label': 'beneficiary'})])

G.add_edges_from(edges[['Provider', 'BeneID']].itertuples(index=False, name=None))

In [ ]:
for component in nx.connected_components(G):
    print(f"Component of size {len(component)}:")

list_components = list(nx.connected_components(G))
largest_component = max(list_components, key=len)

In [ ]:
G = G.subgraph(largest_component).copy()

In [ ]:
# draw network with colourmap according to label of node
ego_net = nx.ego_graph(G, 'PRV51001', radius=3)
node_colors = ['blue' if ego_net.nodes[n]['label'] == 'provider' else 'orange' for n in ego_net.nodes()]
nx.draw_networkx(ego_net, with_labels=False, node_size=50, font_size=5, node_color=node_colors)

## Train-test split

The train-test split can be done in many different ways. Here, we opt to apply the louvain community detection method. The resulting communities are used to split the graph into two groups, one for training and one for testing. 

We set the resolution parameter quite low to favour large communities. 

In [ ]:
communities = nx.community.louvain_communities(G, resolution=0.2, seed = 2025)

In [ ]:
len(communities)

In [ ]:
for com in communities:
    print(f"Community of size: {len(com)}")

In [ ]:
com_1 = [communities[i] for i in [1,2,4]]
com_1_nodes = [node for com in com_1 for node in com]
com_2 = [communities[i] for i in [0,3,5]]
com_2_nodes = [node for com in com_2 for node in com]

In [ ]:
G_train = G.subgraph(com_1_nodes)
G_test = G.subgraph(com_2_nodes)

In [ ]:
dataset_beneficiaries_train = beneficiary_features.loc[[n for n, d in G_train.nodes(data=True) if d['label'] == 'beneficiary']]
dataset_providers_train = provider_features.loc[[n for n, d in G_train.nodes(data=True) if d['label'] == 'provider']]

dataset_beneficiaries_test = beneficiary_features.loc[[n for n, d in G_test.nodes(data=True) if d['label'] == 'beneficiary']]
dataset_providers_test = provider_features.loc[[n for n, d in G_test.nodes(data=True) if d['label'] == 'provider']]

labels_providers_train = labels.loc[dataset_providers_train.index]
labels_providers_test = labels.loc[dataset_providers_test.index]

# Beneficiaries don't have labels. Make dataframes with all 0's
labels_beneficiaries_train = pd.DataFrame(0, index=dataset_beneficiaries_train.index, columns=['Label'])
labels_beneficiaries_test = pd.DataFrame(0, index=dataset_beneficiaries_test.index, columns=['Label'])

In [ ]:
x_train = [
    G_train, 
    dataset_providers_train, 
    dataset_beneficiaries_train, 
    labels_providers_train,
    labels_beneficiaries_train
]

x_test = [
    G_test, 
    dataset_providers_test, 
    dataset_beneficiaries_test, 
    labels_providers_test,
    labels_beneficiaries_test
]

In [ ]:
import pickle
with open('../data/HCP/hcp_train.pkl', 'wb') as f:
    pickle.dump(x_train, f)

with open('../data/HCP/hcp_test.pkl', 'wb') as f:
    pickle.dump(x_test, f)

In [ ]:
import pickle
import networkx as nx
import pandas as pd

with open('../data/HCP/hcp_train.pkl', 'rb') as f:
    hcp_train = pickle.load(f)

with open('../data/HCP/hcp_test.pkl', 'rb') as f:
    hcp_test = pickle.load(f)


In [ ]:
G_train = hcp_train[0]  # Graph
G_test = hcp_test[0]    # Graph

In [ ]:
G_train.number_of_edges()


In [ ]:
G_test.number_of_edges()

In [ ]:
G_train.number_of_nodes()

In [ ]:
G_test.number_of_nodes()

In [ ]:
hcp_train[1]

In [ ]:
hcp_test[2]

In [ ]:
G_deg=G_train.degree()
G_test_deg=G_test.degree()

degree_providers_train = [G_deg[n] for n in G_train.nodes() if G_train.nodes[n]['label'] == 'provider']
degree_providers_test = [G_test_deg[n] for n in G_test.nodes() if G_test.nodes[n]['label'] == 'provider']

In [ ]:
import numpy as np
np.mean(degree_providers_train)


In [ ]:
np.mean(degree_providers_test)

In [ ]:
np.mean([G_deg[n] for n in G_train.nodes()])

In [ ]:
np.mean([G_test_deg[n] for n in G_test.nodes()])

In [ ]:
degree_beneficiaries_train = [G_deg[n] for n in G_train.nodes() if G_train.nodes[n]['label'] == 'beneficiary']
degree_beneficiaries_test = [G_test_deg[n] for n in G_test.nodes() if G_test.nodes[n]['label'] == 'beneficiary']

In [ ]:
np.mean(degree_beneficiaries_train)

In [ ]:
np.mean(degree_beneficiaries_test)

In [ ]:
labels_train = hcp_train[3]; print(labels_train.sum())
labels_test = hcp_test[3]; print(labels_test.sum())

In [ ]:
labels_train = hcp_train[3]; print(labels_train.mean())
labels_test = hcp_test[3]; print(labels_test.mean())

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

X_train = hcp_train[1]
y_train = hcp_train[3]

X_test = hcp_test[1]
y_test = hcp_test[3]

clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict_proba(X_test)[:,1]

from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_test, y_pred); print(roc_auc)

In [ ]:
importance = clf.feature_importances_
feature_names = X_train.columns

forest_importance = pd.Series(importance, index=feature_names)
forest_importance.plot.bar()

In [ ]:
import shap

explainer = shap.TreeExplainer(clf, X_train)
shap_values = explainer(X_test)

shap.summary_plot(shap_values, X_test)